In [ ]:
import sys
import pickle
import numpy as np
import pandas as pd
#from nlp_utils import clean_text, pos_tag_text
sys.path.append("../")
from param_config import config
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
import xgboost as xgb
import datetime

pd.set_option('display.float_format',lambda x: '%.5f'%x)

print("Load data...")

dfTrain = pd.read_csv(config.original_train_data_path)

dfPred= pd.read_csv(config.original_test_data_path)
dfPred2= pd.read_csv(config.original_test_data2_path)
# number of train/test samples
num_train, num_pred = dfTrain.shape[0], dfPred.shape[0]

print("Done.")

f = open('../../Data/col_name.txt','r')  
a = f.read()  
col_name = eval(a)  
f.close() 

f = open('../../Data/Procedure.txt','r')  
a = f.read()  
procedure = eval(a)  
f.close()

f = open('../../Cache/var_change.txt','r')
a = f.read()
var_change = eval(a)
f.close()

dfTrain = dfTrain.rename(columns=col_name)
raw_predictors = dfTrain.columns.tolist()[1:-1]
dfTrain = dfTrain.set_index(dfTrain['ID'])
dfPred = dfPred.rename(columns=col_name)
dfPred = dfPred.set_index(dfPred['ID'])
dfPred2 = dfPred2.rename(columns=col_name)

dfPredAll = pd.concat([dfPred,dfPred2],axis=0)
dfPredAll = dfPredAll.set_index(dfPredAll['ID'])

dfAll = pd.concat([dfTrain,dfPred,dfPred2],axis=0)

In [1]:
import sys
import pickle
import numpy as np
import pandas as pd
#from nlp_utils import clean_text, pos_tag_text
sys.path.append("../")
from param_config import config
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
import xgboost as xgb
import datetime

dfAll = pd.read_csv('../../Data/All_Data_dedup.csv')
dfAll.head(2)

,ID,TOOL_210,TOOL_220,TOOL_300,TOOL_310,TOOL_311,TOOL_312,TOOL_330,TOOL_340,TOOL_344,...,750X1347,750X1350,750X1357,750X1370,750X1380,750X1384,750X1398,750X1408,750X1442,750X1452
0,ID001,N,B,E,E,E0,329,C,3009,W,...,1.0,2.400000e+12,2.07,10.0,10.0,49.9,10.0,10.0,2.400000e+12,2.400000e+12
1,ID002,M,B,E,E,E0,215,C,4106,X,...,1.0,2.400000e+12,2.07,10.0,10.0,50.0,10.0,10.0,2.400000e+12,2.400000e+12


In [2]:
'Y' in dfAll.columns.tolist()

False

In [ ]:
Feat_file = ['Feat_cnt_col','Feat_cnt_tool','Feat_pcent_col','Feat_pcent_tool']
ex_feat = []
for filename in Feat_file:
    tmp = pd.read_csv('../../Cache/'+filename+'.csv')
    tmp.set_index(tmp['ID'],inplace=True,drop=True)
    tmp_feat_list = tmp.columns.tolist()
    if 'ID' in tmp_feat_list:
        tmp_feat_list.remove('ID')
    ex_feat += tmp_feat_list
    dfTrain = pd.merge(dfTrain,tmp,'left','ID')
    dfPred = pd.merge(dfPred,tmp,'left','ID')

In [ ]:
#define the input variables, dummies
predictors = raw_predictors+ex_feat
feat_cnt = 0
for key,value in var_change.items():
    feat_cnt+=1
    if feat_cnt>1:
        continue
    for var in var_change[key]['constant']:
        predictors.remove(var)
    for var in var_change[key]['category']:
        predictors.remove(var)
        tmpTrain = pd.get_dummies(dfTrain[var],prefix=var,dummy_na=True)
        tmpPred = pd.get_dummies(dfPred[var],prefix=var,dummy_na=True)
        predictors = predictors + tmpTrain.columns.tolist()
        dfTrain = pd.concat([dfTrain,tmpTrain],axis=1)
        dfPred = pd.concat([dfPred,tmpPred],axis=1)
for var in predictors:
    if var not in dfPred.columns:
        dfPred[var] = 0

In [ ]:
def xgb_kfold(dfTrain,dfPred,predictors,n_splits=5, params = {'max_depth':3, 'eta':0.01, 'silent':0,'objective':'reg:linear','lambda':1,'subsample':0.8,
                         'colsample_bytree':0.8}):  
    kf = KFold(n_splits=n_splits,shuffle=True)
    dpred = xgb.DMatrix(dfPred[predictors].values,label=[0]*len(dfPred),missing=np.nan)
    imp = pd.DataFrame({'variable':predictors,'lk':['f'+str(i) for i in range(len(predictors))]})
    round=0
    for train_index, test_index in kf.split(dfTrain):
        round+=1
        train_X = dfTrain.loc[train_index,predictors]
        test_X = dfTrain.loc[test_index,predictors]
        train_Y = dfTrain.loc[train_index,'Y']
        test_Y = dfTrain.loc[test_index,'Y']

        dtrain = xgb.DMatrix(train_X.values, label=train_Y.values, missing = np.nan)
        dtest = xgb.DMatrix(test_X.values, label=test_Y.values, missing = np.nan)
        param = params 
        evallist  = [(dtrain,'train'),(dtest,'eval')]  
        num_round = 5000
        evals_dict = {}
        model = xgb.train(param,dtrain,num_round, evallist,early_stopping_rounds=50,evals_result=evals_dict,verbose_eval =1000)
        performance_df = pd.DataFrame(evals_dict['eval'])
        bst_tree = len(performance_df)-51
        pred_test = model.predict(dtest,ntree_limit =bst_tree)

        tmp_imp = pd.DataFrame(model.get_score(importance_type='gain'),index=['imp_fold%d'%round]).T
        tmp_imp['lk'] = tmp_imp.index
        imp = imp.merge(tmp_imp,'left','lk').fillna(0)


        pred_score = model.predict(dpred,ntree_limit =bst_tree)
        if round==1:
            test_result = pd.DataFrame({'ID':dfTrain.loc[test_index,'ID'].values,'score':pred_test,'target':test_Y})
            result = pd.DataFrame({'ID':dfPred['ID'],'Score_%d'%round:pred_score})
        else:
            test_result = pd.concat([test_result,pd.DataFrame({'ID':dfTrain.loc[test_index,'ID'].values,'score':pred_test,'target':test_Y})],axis=0)
            result = result.merge(pd.DataFrame({'ID':dfPred['ID'],'Score_%d'%round:pred_score}),'inner','ID')
    print("Test MSE:",metrics.mean_squared_error(test_result['target'], test_result['score']))
    return test_result,result,imp

In [ ]:
test_result,result,imp = xgb_kfold(dfTrain,dfPred,predictors)

In [ ]:
result['score']=result[['Score_1','Score_2','Score_3','Score_4','Score_5']].mean(axis=1)
submit = result[['ID','score']]
today = datetime.date.today().strftime('%Y-%m-%d')
submit.to_csv('../../Submission/submit_%s.csv'%today,header=False,index=False)